In [1]:
import cv2
import numpy as np
import os, sys
import time
import math

def toDHash(img):
    img = cv2.resize(img, (9, 8), interpolation=cv2.INTER_CUBIC)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    h = []
    for i in range(8):
        for j in range(8):
            if gray[i, j] > gray[i, j+1]:
                h.append(1)
            else:
                h.append(0)
    return h

# hamming distance
def hDist(hash1, hash2):
    num = 0
    for idx in range(len(hash1)):
        if hash1[idx] != hash2[idx]:
            num += 1
    return num

def dHashDiff(i1, i2):
    h1 = toDHash(i1)
    h2 = toDHash(i2)
    return hDist(h1, h2)

def convertVideoToImage(video_paths, save_title, save_root=None, crop_size=None):
    start_time = time.time()
    
    if save_root is None:
        save_root = save_title
    save_path = './imgs/from_video/' + save_root
    os.mkdir(save_path)
    
    fps_fix = 3 #fps15，fps_fix取3，一秒5張
    dim = (720, 405)
    # jpeg_flag = [int(cv2.IMWRITE_JPEG_QUALITY), 50]
    diff_threshold = 0
    
    subDir = ''
    count = -1
    img_count = 0
    subDir_flag = True
    
    for video_path in video_paths:
        cam = cv2.VideoCapture(video_path)
        start_flag = False
        ret_val = True

        pre_frame = None

        while ret_val:
            ret_val, image = cam.read()
            count = count + 1
            if count % fps_fix != 0:
                continue;

            if img_count % 100 == 0 and subDir_flag:
                subDir = '/img_' + str(img_count) + '-' + str(img_count + 99)
                os.mkdir(save_path + subDir)
                subDir_flag = False

            if not start_flag:
                if image is None:
                    print("image \"%s\" is not exist" % video_path)
                    break;
                print('video image: (%d, %d)' % (image.shape[1], image.shape[0]))
                start_flag = True
            if image is not None:
                image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
                # image = cv2.GaussianBlur(image, (31, 31), 0)
                # crop
                if crop_size is not None:                
                    (x1, y1, x2, y2) = crop_size
                    if x2 is None:
                        x2 = image.shape[1]
                    if y2 is None:
                        y2 = image.shape[0]
                    # crop
                    image = image[y1:y2, x1:x2]
                if pre_frame is None or dHashDiff(image, pre_frame) > diff_threshold:
                    # _path = (save_path + subDir + '/' + save_title + '-%d.jpg') % img_count
                    # cv2.imwrite(_path, image, jpeg_flag)
                    _path = (save_path + subDir + '/' + save_title + '-%d.png') % img_count
                    # print('save image at: ' + _path)
                    cv2.imwrite(_path, image)

                    img_count = img_count + 1
                    subDir_flag = True
                    pre_frame = image
            else:
                print('frame is empty...')

            if cv2.waitKey(1) == 27:
                break
    print('Finish in %.4f secs.' % (time.time() - start_time))
    print('Number of images: %d/%d' % (img_count, math.floor(count/fps_fix) + 1))
    cv2.destroyAllWindows()

In [2]:
# convertVideoToImage('./mdata/videos/test_data_2.mp4', './mdata/images/from_video/test_data_2_w720h405q50')
paths = ['./mdata/videos/bed/data_1.mp4', './mdata/videos/bed/data_2.mp4']
convertVideoToImage(paths, './imgs/from_video/bed_data_1')
# (120, 0, None, None)

video image=1920x1080
frame is empty...
